In [1]:
from keras.layers import Input, Dense, Conv2D, Reshape, MaxPooling2D, UpSampling2D, Flatten, BatchNormalization, Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard

class FeatureExtractor:
    def __init__(self):
        self.encoder_model = None
        self.model = None
        return
    
    def build(self, input_dims, opt):
        input_layer = Input(shape=input_dims)
        
        a_one = Conv2D(64, (3,3), activation='relu', padding='same') (input_layer)
        #a_two = BatchNormalization() (a_one)
        a_three = Conv2D(64, (3,3), activation='relu', padding='same') (a_one)
        #a_four = BatchNormalization() (a_three)
        a_five = MaxPooling2D() (a_three)
        block_one = Dropout(0.25) (a_five)
        #block_one = a_five
        
        b_one = Conv2D(128, (3,3), activation='relu', padding='same') (block_one)
        #b_two = BatchNormalization() (b_one)
        b_three = Conv2D(128, (3,3), activation='relu', padding='same') (b_one)
        #b_four = BatchNormalization() (b_two)
        b_five = MaxPooling2D() (b_three)
        block_two = Dropout(0.25) (b_five)
        
        c_one = Conv2D(256, (3,3), activation='relu', padding='same') (block_two)
        #c_two = BatchNormalization() (c_one)
        c_three = Conv2D(256, (3,3), activation='relu', padding='same') (c_one)
        #c_four = BatchNormalization() (c_three)
        c_five = MaxPooling2D() (c_three)
        block_three = Dropout(0.5) (c_five)
        
        d_one = Conv2D(512, (3,3), activation='relu', padding='same') (block_three)
        #d_two = BatchNormalization() (d_one)
        d_three = Conv2D(512, (1,1), activation='relu', padding='same') (d_one)
        #d_four = BatchNormalization() (d_three)
        d_five = MaxPooling2D() (d_three)
        block_four = Dropout(0.2) (d_five)
        
        flat = Flatten() (block_four)
        fc_one = Dense(4096, activation='relu') (flat)
        #block_five = BatchNormalization() (fc_one)
        
        fc_two = Dense(4096, activation='relu') (fc_one)
        #block_six = BatchNormalization() (fc_two)
        
        final = Dense(4, activation='softmax') (fc_two)
        
        self.model = Model(input_layer, final)
        self.feature_extractor = Model(input_layer, flat)
        self.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        return
    
    def load(self, model_file, encoder_model_file):
        self.encoder_model = load_model(encoder_model_file)
        self.model = load_model(model_file)
        return
    
    def train(self, train_input, train_output,
             val_input, val_output,
             epochs=50,
             batch_size=64,
             shuffle=True):
        tensorboard = TensorBoard(log_dir='./tf_logs', histogram_freq=0, write_graph=True, write_images=False)
        self.model.fit(train_input, train_output,
                      epochs=epochs, batch_size=batch_size,
                      shuffle=shuffle,
                      validation_data=(val_input, val_output),
                      callbacks=[tensorboard])
        return
    
    def encoder_predict(self, test_input):
        return self.encoder_model.predict(test_input)
    
    def predict(self, test_input):
        return self.model.predict(test_input)
    
    def save(self, model_file, encoder_model_file):
        self.model.save(model_file)
        self.encoder_model.save(encoder_model_file)
        return

Using TensorFlow backend.


In [2]:
from keras.datasets import cifar10
from keras import optimizers
import numpy as np
from sklearn import preprocessing


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
interested = [0, 1, 8, 9]

scrap = []
for idx, im in enumerate(x_train):
    if (y_train[idx][0] not in interested):
        scrap.append(idx)
        
x_train = np.delete(x_train, scrap, axis=0)
y_train = np.delete(y_train, scrap, axis=0)

enc = preprocessing.OneHotEncoder()
enc.fit(y_train)
y_train = enc.transform(y_train).toarray()

scrap = []
for idx, im in enumerate(x_test):
    if (y_test[idx][0] not in interested):
        scrap.append(idx)
x_test = np.delete(x_test, scrap, axis=0)
y_test = np.delete(y_test, scrap, axis=0)
y_test = enc.transform(y_test).toarray()

x_train = (x_train.astype('float32')) / 255.0
x_test = (x_test.astype('float32')) / 255.0

print x_train.shape
print x_test.shape

(20000, 32, 32, 3)
(4000, 32, 32, 3)


In [ ]:
fe = FeatureExtractor()
opt = optimizers.adam(lr=0.0001, decay=1e-6)
#opt = optimizers.rmsprop()
#opt = optimizers.SGD(lr=0.1, nesterov=True, momentum=0.9)
fe.build((32, 32, 3, ), opt)

In [ ]:
fe.train(x_train, y_train, x_test, y_test,
                 epochs=100,
                 batch_size=128,
                 shuffle=True)

Train on 20000 samples, validate on 4000 samples
Epoch 1/100
20000/20000 [==============================] - 80s - loss: 1.2374 - acc: 0.3981 - val_loss: 1.0555 - val_acc: 0.5413
Epoch 2/100
20000/20000 [==============================] - 77s - loss: 1.0317 - acc: 0.5354 - val_loss: 0.9526 - val_acc: 0.6102
Epoch 3/100
20000/20000 [==============================] - 77s - loss: 0.9014 - acc: 0.6035 - val_loss: 0.7927 - val_acc: 0.6783
Epoch 4/100
20000/20000 [==============================] - 77s - loss: 0.7704 - acc: 0.6835 - val_loss: 0.7420 - val_acc: 0.6917
Epoch 5/100
20000/20000 [==============================] - 77s - loss: 0.6485 - acc: 0.7446 - val_loss: 0.5632 - val_acc: 0.7865
Epoch 6/100
20000/20000 [==============================] - 77s - loss: 0.5620 - acc: 0.7847 - val_loss: 0.4957 - val_acc: 0.8133
Epoch 7/100
20000/20000 [==============================] - 77s - loss: 0.4844 - acc: 0.8179 - val_loss: 0.4462 - val_acc: 0.8307
Epoch 8/100
20000/20000 [=======================

KeyboardInterrupt: 

In [ ]:
fe.save('extractor.h5', 'extractor-model.h5')
print "Model saved!"

Model saved!
